# The Kevin Bacon game, kind of

Use this notebook to see whether {film person} has ever done a movie with {other film person}. [Uses data from IMDB](https://datasets.imdbws.com/).

If you haven't already downloaded the data (three gzipped .tsv files, updated nightly), uncomment and run this cell:

In [30]:
# %run -i download.py

Then import pandas:

In [31]:
import pandas as pd

Drop in the names of the people you're looking for as strings:

In [32]:
person1 = 'Christian Bale'
person2 = 'Werner Herzog'

Read in the name data:

In [33]:
names = pd.read_csv('data/name.basics.tsv.gz',
                    compression='gzip',
                    sep='\t',
                    na_values='\\N')

In [13]:
names.head()

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,nm0000001,Fred Astaire,1899.0,1987.0,"soundtrack,actor,miscellaneous","tt0072308,tt0053137,tt0043044,tt0050419"
1,nm0000002,Lauren Bacall,1924.0,2014.0,"actress,soundtrack","tt0071877,tt0037382,tt0038355,tt0117057"
2,nm0000003,Brigitte Bardot,1934.0,NaN,"actress,soundtrack,producer","tt0059956,tt0057345,tt0054452,tt0049189"
3,nm0000004,John Belushi,1949.0,1982.0,"actor,writer,soundtrack","tt0080455,tt0078723,tt0077975,tt0072562"
4,nm0000005,Ingmar Bergman,1918.0,2007.0,"writer,director,actor","tt0050976,tt0083922,tt0069467,tt0050986"


Next steps involve some human judgement:
- Filter the data to show just records where the `primaryName` contains either of the names defined in the `person1` and `person2` variables
- When you've found the correct records, look at the unique person ID in the `nconst` column and assign each ID number (as a string) to a new variable (`person1_id` and `person2_id`

In [35]:
# filter the names data to show records that include the names
# defined in either the `person1` or `person2` variables
# -- Christian Bale and Werner Herzog, in this case
# ... the v clever idea to use a | to join the list and do a regex OR search
# came from https://stackoverflow.com/a/26577689
names[names.primaryName.str.contains('|'.join([person1, person2]), case=False)]

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
287,nm0000288,Christian Bale,1974.0,NaN,"actor,editorial_department,producer","tt0468569,tt0372784,tt1800241,tt6266538"
1346,nm0001348,Werner Herzog,1942.0,NaN,"director,writer,actor","tt0083946,tt0790724,tt0427312,tt0068182"
3703879,nm3577667,Christian Bales,NaN,NaN,actor,"tt0822831,tt5185564"
7162959,nm7635250,Christian Balenciaga,NaN,NaN,make_up_department,tt1252370


In [36]:
# we want Christian Bale, not Christian Bales (!) of Christian Balenciaga,
# so our horse is nm0000288
person1_id = 'nm0000288'

# and Werner is nm0001348
person2_id = 'nm0001348'

Now we can read in the principles data:

In [18]:
principles = pd.read_csv('data/title.principals.tsv.gz',
                         compression='gzip',
                         sep='\t',
                         na_values='\\N')

In [19]:
principles.head()

,tconst,ordering,nconst,category,job,characters
0,tt0000001,1,nm1588970,self,NaN,"[""Herself""]"
1,tt0000001,2,nm0005690,director,NaN,NaN
2,tt0000001,3,nm0374658,cinematographer,director of photography,NaN
3,tt0000002,1,nm0721526,director,NaN,NaN
4,tt0000002,2,nm1335271,composer,NaN,NaN


Filter to get just the roles involving either of our people of interest:

In [29]:
filtered_data = principles[principles.nconst.isin([person1_id, person2_id])]

Now get a list of duplicate title IDs:

In [21]:
matches = filtered_data[filtered_data.duplicated(subset='tconst')]

In [22]:
matches

,tconst,ordering,nconst,category,job,characters
3221867,tt0462504,5,nm0001348,director,NaN,NaN
10144843,tt1454425,4,nm0001348,self,NaN,"[""Himself""]"


Now read in the title data:

In [24]:
titles = pd.read_csv('data/title.basics.tsv.gz', 
                     compression='gzip',
                     sep='\t',
                     na_values='\\N')

/Users/cjwinchester/.virtualenvs/movie-match-CfsPrWWZ/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3044: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [25]:
titles.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894.0,NaN,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892.0,NaN,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892.0,NaN,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892.0,NaN,NaN,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893.0,NaN,1,"Comedy,Short"


Finally, filter the title data to include only titles that match the `tconst` IDs in the `matches` data frame.

In [26]:
titles[titles.tconst.isin(matches.tconst)]

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
444989,tt0462504,movie,Rescue Dawn,Rescue Dawn,0,2006.0,NaN,120,"Action,Adventure,Biography"
1405287,tt1454425,tvEpisode,Episode dated 10 July 2007,Episode dated 10 July 2007,0,2007.0,NaN,NaN,Talk-Show


[brb going to the library](https://pikp.ent.sirsi.net/client/en_US/PPLD/search/detailnonmodal/ent:$002f$002fSD_ILS$002f0$002fSD_ILS:690883/email?d=ent%3A%2F%2FSD_ILS%2F0%2FSD_ILS%3A690883~ILS~0)